## Vessels passing a lock in a real-world example
In this example we will discuss how to implement an existing lock from real-world data in OpenTNSim and how to analyse specific locking output.

We take the following steps:

1. [Imports](#1-imports)
2. [Create graph](#2-create-graph)
3. [Create locks](#3-create-locks)
4. [Create vessel](#4-create-vessel)
4. [Run simulation](#5-run-simulation)
5. [Inspect output](#6-inspect-ouput)

### 1. Imports

Import the required libraries

In [1]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {}'.format(opentnsim.__version__))

This notebook has been tested with OpenTNSim version 1.1.2


In [2]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib
import warnings

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style
import folium

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr, osr

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot

# Used for mathematical functions
import math             
import json

# Used for making the graph to visualize our problem
import networkx as nx 

from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 


In [3]:
# Graph location
src_dir = pathlib.Path(opentnsim.__file__).parent.parent

# Graph location
location_graph = src_dir / "notebooks"
name_graph = location_graph / "Shape-Files" / "Vaarwegvakken" / "Vaarwegvakken.shp"

In [4]:
# Start simpy environment
# env = simpy.Environment()
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

### 2. Create graph

The cel below visualizes the problem. In graph theory the red dots are called *edges* and the lines are called *vertices*. Vessels (or any other object) move from edge 1 to edge 3 and from edge 4 to edge 2. The added complexity is that vertice 5-6 only allows traffic in one direction at a time. Vessels can travel simultanously in one direction.

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

The shapefile `Vaarwegvakken.shp` is used as a basis for creating the graph for this simulation. To make things easier, the directory (location) of the shapefile is expressed as the variable `location_graph`. The name of the file is expressed as the variable `name_graph`. This will be used troughout the rest of the code. 

In [5]:
# Create a directed graph with single edges using read_shp
FG = nx.read_shp(str(name_graph), simplify=True)

# We require a directed graph but two-way traffic

FG = FG.to_undirected()
FG = FG.to_directed()

/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_83070/1758183640.py:2: DeprecationWarning:

read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.



#### Convert to WGS84

WGS84 is the latest version of the World Geodetic System. More information can be found [here](https://en.wikipedia.org/wiki/World_Geodetic_System).

In [6]:
def transform_projection(location_graph, name_graph):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataset = driver.Open(os.path.join(location_graph, name_graph))

    # from Layer
    inSpatialRef = dataset.GetLayer().GetSpatialRef()

    # Set up the coordinate reference we want to use, WGS84 - World Geodetic System 1984
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)

    # Transform the coordinates
    transform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    return transform

In [7]:
def change_projection(transform, point):
    point = ogr.CreateGeometryFromWkt(str(point))
    
    point.Transform(transform)
    point.ExportToWkt()
    
    return point.GetX(), point.GetY()

In [8]:
transform = transform_projection(location_graph, name_graph)
FG_new = nx.DiGraph()

nodes_dict = {}

for i, node in enumerate(FG.nodes(data = True)):
    coordinates = change_projection(transform, shapely.geometry.Point(list(FG.nodes)[i][0], list(FG.nodes)[i][1]))
    name = "({:f}, {:f})".format(coordinates[0], coordinates[1])
    geometry = shapely.geometry.Point(coordinates[0], coordinates[1])
    
    nodes_dict[list(FG.nodes)[i]] = name
    FG_new.add_node(name, name = name, Position = coordinates, geometry = geometry, Old = node[1])
    
for edge in FG.edges(data = True):
    node_1 = nodes_dict[edge[0]]
    node_2 = nodes_dict[edge[1]]
    
    VRT_NAAM = edge[2]["VRT_NAAM"]
    VWG_NAAM = edge[2]["VWG_NAAM"]
    BEGKM =  edge[2]["BEGKM"]
    ENDKM =  edge[2]["ENDKM"]
    DIST = np.abs(float(BEGKM) - float(ENDKM))
    
    LINE = (json.loads(edge[2]["Json"])["coordinates"])
    LineString = []
    for coordinates in LINE:
        LineString.append(change_projection(transform, shapely.geometry.Point(coordinates[0], coordinates[1])))
    
    FG_new.add_edge(node_1, node_2, LineString = shapely.geometry.LineString(LineString), 
                    VRT_NAAM = VRT_NAAM, VWG_NAAM = VWG_NAAM, BEGKM = BEGKM, ENDKM = ENDKM, DIST = DIST)

if nx.info(FG) == nx.info(FG_new):
    print("Succes!")

Succes!


#### Select only relevant area

In [9]:
# North-East
NE = (4.54, 51.75)
# South-East
SE = (4.54, 51.60)
# South-West
SW = (4.20, 51.60)
# North-West
NW = (4.20, 51.75)

polygon = shapely.geometry.Polygon([NE, SE, SW, NW])

In [10]:
nodes = []
edges = []

for edge in FG_new.edges(data = True):
    node_1 = FG_new.nodes[edge[0]]
    node_2 = FG_new.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        edges.append(edge)

In [11]:
FG_new = nx.DiGraph ()

for node in nodes:
    FG_new.add_node(node["name"], name = node["name"], Position = node["Position"], geometry = node["geometry"])

for edge in edges:
    FG_new.add_edge(edge[0], edge[1], Info = edge[2])

#### Show on map

In [12]:
# Browser
m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

for edge in FG_new.edges(data = True):
    points_x = list(edge[2]["Info"]["LineString"].coords.xy[0])
    points_y = list(edge[2]["Info"]["LineString"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Info"]["VWG_NAAM"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)
    
    else:
        folium.PolyLine(line, weight = 2, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)

m

### 3. Create locks
We can see on the maps that there are three locks on the graph, but that the information on the locks is limited. The following edges represent locks:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [13]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

TypeError: __init__() missing 7 required positional arguments: 'node_3', 'wlev_dif', 'disch_coeff', 'grav_acc', 'opening_area', 'opening_depth', and 'simulation_start'

In [ ]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]

### 4. Create vessel

In [ ]:
Vessel = type('Vessel', 
              (core.Identifiable, core.Movable, core.HasContainer, core.HasResource, core.Routeable), {})

data_vessel_one = {"env": env,
                   "name": "Vessel",
                   "route": ["(4.430289, 51.700047)", "(4.392555, 51.681251)"],
                   "geometry": nx.get_node_attributes(FG_new, "geometry")["(4.430289, 51.700047)"],
                   "capacity": 1_000}

data_vessel_two = {"env": env,
                   "name": "Vessel",
                   "route": ["(4.392555, 51.681251)", "(4.430289, 51.700047)"],
                   "geometry": nx.get_node_attributes(FG_new, "geometry")["(4.392555, 51.681251)"],
                   "capacity": 1_000}

### 5. Run simulation

In [ ]:
# Start simpy environment
env.FG = FG_new

# Add the movements of the vessel to the simulation
vessels = []
for i in range(10):
    vessel = Vessel(**data_vessel_one)
    vessels.append(vessel)
    env.process(vessel.move())
    
    vessel = Vessel(**data_vessel_two)
    vessels.append(vessel)
    env.process(vessel.move())

# Run the environment
env.run()

### 6. Inspect ouput

In [ ]:
# First ship should be able to directly enter the lock
pd.DataFrame.from_dict(vessels[0].log)

In [ ]:
# Second ship should be able to pass the lock after the first one has passed
pd.DataFrame.from_dict(vessels[1].log)

In [ ]:
# Check what the lock is doing
pd.DataFrame.from_dict(lock_nr_1.log).head(10)

In [ ]:
# Fourth ship should be the first one the start waiting at the line-up area
pd.DataFrame.from_dict(vessels[3].log)

In [ ]:
# Seventh ship should be the first one the start waiting at the waiting area
pd.DataFrame.from_dict(vessels[6].log)

In [ ]:
# Tenth ship should be the first one the start waiting to enter the waiting area
pd.DataFrame.from_dict(vessels[9].log)